# Process an EDN CSV file to generate a high detail, smoothed .cube LUT file
This notebook allows you to process a CSV file generated by the Easy Digital Negatives (EDN) site into a LUT curve .cube file to use in making alternative process digital negatives.  This process allows you to generate much more detailed .cube files, and the number of steps will match the number of grey patches in the step tablet image you used to make your test.  If, for example, you used the 256-step image to produce your negative, the output curve this notebook will generate will have 256 steps.

**If you already have a .cube file obtained from Easy Digital Negatives, and you want to smooth it, you should use the Cube_File_Smoothing notebook instead of this one.**

### How To Run This Notebook
The code in this notebook is organized into cells.  Each cell can be run independently, providing all prerequisite imports have been done and all variables exist.  However, **it is recommended that you run all code cells in sequence.**  

The only exception is possibly the [Smooth the Data](#Smooth-the-data) cell, which you may choose to run more than once if you wish to revise your smoothing factor prior to saving the new .cube file.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from tkinter import Tk
from tkinter import filedialog
from tkinter import simpledialog

In [ ]:
root = Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()
# get the Easy Digital Negatives CSV file to process
input_csv = filedialog.askopenfilename(title="Easy Digital Negatives CSV data file", filetypes=[("CSV files", "*.csv")])
# get the name of the output .cube file
cube_filename = simpledialog.askstring("Output .cube File Name", "Enter the name for the output .cube file (no .cube extension)")
cube_filename = cube_filename.replace(" ", "_")
print("Processing CSV file: {}".format(input_csv))
print("\t to make {}.cube".format(cube_filename))

In [ ]:
def interpolate(xval, df, xcol, ycol):
    """
    This function returns an interpolated value from a dataframe at a specified X value.
    
    Inputs
    ------
    xval: numeric - the X value at which you wish to get the interpolated Y value
    df: pandas dataframe - the dataframe containing the data to be interpolated
    xcol: string - the name of the column in the dataframe containing X values
    ycol: string - the name of the column in the dataframe containing Y values
    
    Returns
    -------
    (interpolated_value, X, Y)
    """
    return np.interp([xval], df[xcol], df[ycol])

In [ ]:
# read the CSV file from Easy Digital Negatives
csv_data = pd.read_csv(input_csv, sep='\t')
csv_data.rename(columns={"Idea":"Linear", "Scan":"Scan"}, inplace=True)
csv_vals = []
for i in range(0, len(csv_data)):
    csv_vals.append(float(i))

# Smooth the data
The following cell smooths the data using a Univariate Spline with a smoothing factor.  You will be prompted to specify the smoothing factor.  Common values fall within the 500 to 12000 range, and the higher this number, the smoother the resulting curve.  

**If, after running this cell, you do not like the resulting smoothed curve, you may re-run this cell as many times as needed to narrow down the smoothing factor you like best.**

In [ ]:
# get the smoothing factor
smoothing_factor_value = simpledialog.askinteger(title="Smoothing Factor", prompt="Enter a smoothing factor.\nBigger numbers mean a smoother result.\nValues between 500 and 12000 are most common.", initialvalue=800)
new_curve_vals = []
new_df = pd.DataFrame({"Curve":csv_vals, 'Values':list(csv_data['Scan'])}, index=csv_data['Scan'])
# mirror the scan data around the diagonal to create the curve
for i in csv_vals:
    ival = interpolate(i, new_df, 'Values', 'Curve')
    new_curve_vals.append(ival[0])
csv_data['Curve'] = new_curve_vals
# smooth the curve
csv_spline = UnivariateSpline(csv_vals, csv_data['Curve'], k=4)
csv_spline.set_smoothing_factor(smoothing_factor_value)
csv_data['Smoothed_Curve'] = csv_spline(csv_data['Linear'])
# plot the dataframe
plt.close("all")
csv_data.plot(figsize=(7,7), grid=True)
plt.xlabel('Data Step')
plt.ylabel('8-bit Value')
title_text = "{}\nSmoothing Factor:{}".format(cube_filename, smoothing_factor_value)
plt.title(title_text)
plt.show()
# IF THE SMOOTHING FACTOR NEEDS ADJUSTMENT, RE-RUN THIS CELL WITH DIFFERENT SMOOTHING VALUES UNTIL YOU ARE SATISFIED

# Save the .cube file
After you are satisfied with the smoothed curve in the cell above, use the cell below to save your output .cube file.  It will use the name you provided earlier, and will be stored in the same folder as the .csv file you chose.

In [ ]:
# convert the curve so it falls within 0.0 and 1.0
converted_indices = [round(e/max(csv_vals),3) for e in csv_vals]
converted_curve = [round(e/max(csv_vals),6) for e in list(csv_data['Smoothed_Curve'])]
output_df = pd.DataFrame({'Linear':converted_indices, 'Curve':converted_curve})

# save the new curve to a .cube file
out_cube = os.path.join(os.path.dirname(input_csv), "{}.cube".format(cube_filename))
with open(out_cube, 'w') as cube:
    cube.write("#Created by: Easy Digital Negatives plus Jupyter\nTITLE  \"{}\"\nLUT_1D_SIZE {}\nDOMAIN_MIN 0.0 0.0 0.0\nDOMAIN_MAX 1.0 1.0 1.0\n#LUT data points\n".format(cube_filename, len(converted_curve)))
    line_template = "{:<08} {:<08} {:<08}\n"
    for i, row in output_df.iterrows():
        if row['Curve'] < 0.0:
            val = 0.0
        elif row['Curve'] > 1.0:
            val = 1.0
        else:
            val = round(row['Curve'], 6)
        cube.write(line_template.format(val, val, val))
    cube.write("#END data\n")